# Parameters

In [53]:
dataset_metatada = './networks/metadata.json'
opt = 'selected'
dvs = {
    "selected": {
        "TopologyZoo": ["Bellcanada", "Cogentco"],
        "DEFO": ["rf1239_real_hard"],
        "FatTree": ["FatTree8", "FatTree16", "FatTree32"]
    }
}
dataset_view = dvs[opt] if opt in dvs else None

# Module Imports

In [54]:
import collections
import json
import math
import random
import time

import networkx as nx
import numpy as np
import pandas as pd

In [55]:
# MATPLOTLIB AND SEABORN
import matplotlib.pylab as pylab
from matplotlib.patches import Rectangle
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

%matplotlib inline

pylab.rcParams['font.size'] = 24
# pylab.rcParams['figure.figsize'] = [3*x for x in [3.52, 2.64]]
pylab.rcParams['figure.figsize'] = [2.5*x for x in [4.40, 2.75]]
pylab.rcParams['figure.dpi'] = 100
pylab.rcParams['errorbar.capsize'] = 3
pylab.rcParams['legend.fontsize'] = 12
pylab.rcParams['lines.linewidth'] = 2

gridcolor = '#bbbbbb'

zvalue = 2.576  # 99%

# Processing Functions

In [56]:
def proc_network(cmd, cdf, tgt_consts):
    # Network-wise constants
    words_per_nft_args_entry = int(math.ceil(cmd['nrml_fwding_table_args_bitwidth']/tgt_consts['sram_word_bitwidth']))
    words_per_aft_key_entry = int(math.ceil(cmd['alt_fwding_table_key_bitwidth']/tgt_consts['sram_hashtable_key_bitwidth']))
    words_per_aft_args_entry = int(math.ceil(cmd['alt_fwding_table_args_bitwidth']/tgt_consts['sram_word_bitwidth']))
    words_per_ot_key_entry = int(math.ceil(cmd['opposite_table_key_bitwidth']/tgt_consts['sram_hashtable_key_bitwidth']))
    words_per_ot_args_entry = int(math.ceil(cmd['opposite_table_args_bitwidth']/tgt_consts['sram_word_bitwidth']))
    words_per_stt_key_entry = int(math.ceil(cmd['state_transition_table_key_bitwidth']/tgt_consts['sram_hashtable_key_bitwidth']))
    words_per_stt_args_entry = int(math.ceil(cmd['state_transition_table_args_bitwidth']/tgt_consts['sram_word_bitwidth']))
    #
    nft_args_sram_extra_words = (words_per_nft_args_entry - 1)*int(cdf['n_nrml_fwding_table_entries'].max())
    nft_args_sram_extra_blocks = int(math.ceil(nft_args_sram_extra_words/tgt_consts['sram_words_per_block']))
    aft_key_sram_words = words_per_aft_key_entry*int(cdf['n_alt_fwding_table_entries'].max())
    aft_key_sram_blocks = int(math.ceil(aft_key_sram_words/tgt_consts['sram_words_per_block']))
    aft_args_sram_words = words_per_aft_args_entry*int(cdf['n_alt_fwding_table_entries'].max())
    aft_args_sram_blocks = int(math.ceil(aft_args_sram_words/tgt_consts['sram_words_per_block']))
    ot_key_sram_words = words_per_ot_key_entry*int(cdf['n_opposite_table_entries'].max())
    ot_key_sram_blocks = int(math.ceil(ot_key_sram_words/tgt_consts['sram_words_per_block']))
    ot_args_sram_words = words_per_ot_args_entry*int(cdf['n_opposite_table_entries'].max())
    ot_args_sram_blocks = int(math.ceil(ot_args_sram_words/tgt_consts['sram_words_per_block']))
    stt_key_sram_words = words_per_stt_key_entry*int(cdf['n_state_transition_table_entries'].max())
    stt_key_sram_blocks = int(math.ceil(stt_key_sram_words/tgt_consts['sram_words_per_block']))
    stt_args_sram_words = words_per_stt_args_entry*int(cdf['n_state_transition_table_entries'].max())
    stt_args_sram_blocks = int(math.ceil(stt_args_sram_words/tgt_consts['sram_words_per_block']))
    #

    return [
        nft_args_sram_extra_words, nft_args_sram_extra_blocks,
        aft_key_sram_words, aft_key_sram_blocks, aft_args_sram_words, aft_args_sram_blocks,
        ot_key_sram_words, ot_key_sram_blocks, ot_args_sram_words, ot_args_sram_blocks,
        stt_key_sram_words, stt_key_sram_blocks, stt_args_sram_words, stt_args_sram_blocks,
        round(cmd["ttc_normal_forwarding"], 3), round(cmd["ttc_alt_forwarding"], 3),
        cdf["n_alt_fwding_table_entries"].sum(), cdf["n_alt_fwding_table_entries"].max(),
        cmd["n_nodes"], cmd["n_edges"]
    ]

In [57]:
def main_loop(dm, tgt_consts):
    base_dir = './results/routing/summary'
    datasets = dm['datasets']

    res_cols = [
        'nft_args_sram_extra_words', 'nft_args_sram_extra_blocks',
        'aft_key_sram_words', 'aft_key_sram_blocks', 'aft_args_sram_words', 'aft_args_sram_blocks',
        'ot_key_sram_words', 'ot_key_sram_blocks', 'ot_args_sram_words', 'ot_args_sram_blocks',
        'stt_key_sram_words', 'stt_key_sram_blocks', 'stt_args_sram_words', 'stt_args_sram_blocks',
        "ttc_normal_forwarding", "ttc_alt_forwarding",
        "n_total_alt_forwarding_entries", "n_alt_forwarding_entries",
        "n_nodes", "n_edges",
    ]
    df = pd.DataFrame(columns=res_cols)

    ps = {
        "gm": {},
        "rm": {}
    }

    for ds in datasets:
        print(f"Dataset={ds['name']}")
        for nn in ds['networks']:
            print(f"  Network={nn}")
            with open(f"{base_dir}/{ds['name']}-{nn}.json") as f:
                cmd = json.load(f)
            cdf = pd.read_csv(f"{base_dir}/{ds['name']}-{nn}.csv", index_col=0)
            
            res = proc_network(cmd, cdf, tgt_consts)
            
            # print(cmd)
            # display(cdf)
            df.loc[nn] = res
    return df, ps


# Compute Data

In [58]:
%%time

# Open full metadata
with open(dataset_metatada) as f:
    dm = json.load(f)

# Apply view
if dataset_view is not None:
    datasets = []
    for ds in dm['datasets']:
        if ds['name'] in dataset_view.keys():
            ds['networks'] = dataset_view[ds['name']]
            datasets.append(ds)
    dm['datasets'] = datasets

# RMT Constants
rmt_constants = {
    'stages': 32,
    'sram_blocks_per_stage': 106,
    'sram_total_blocks': 32*106,
    'sram_words_per_block': 1024,
    'sram_word_bitwidth': 96,
    'sram_hashtable_key_bitwidth': 80,
    'sram_block_size_bits': 112*1024,
    'tcam_blocks_per_stage': 16,
    'tcam_total_blocks': 32*16,
    'tcam_key_bitwidth': 40,
    'tcam_words_per_block': 2048
}

# Run main loop
df, ps = main_loop(dm, rmt_constants)

Dataset=TopologyZoo
  Network=Bellcanada
  Network=Cogentco
Dataset=DEFO
  Network=rf1239_real_hard
Dataset=FatTree
  Network=FatTree8
  Network=FatTree16
  Network=FatTree32
CPU times: user 57.3 ms, sys: 5.21 ms, total: 62.5 ms
Wall time: 61.6 ms


In [59]:
df['total_sram_blocks'] = df["nft_args_sram_extra_blocks"] + df["aft_key_sram_blocks"] + df["aft_args_sram_blocks"] + df["ot_key_sram_blocks"] + df["ot_args_sram_blocks"] + df["stt_key_sram_blocks"] + df["stt_args_sram_blocks"]
df['total_sram_Kbits'] = df['total_sram_blocks']*rmt_constants['sram_block_size_bits']/1024
df['total_sram_block_util'] = (df['total_sram_blocks']*100/rmt_constants['sram_total_blocks']).apply(lambda x: round(x, 2))
df['total_sram_words'] = df["nft_args_sram_extra_words"] + df["aft_key_sram_words"] + df["aft_args_sram_words"] + df["ot_key_sram_words"] + df["ot_args_sram_words"] + df["stt_key_sram_words"] + df["stt_args_sram_words"]

In [60]:
ns = {
    "FatTree8": "FT8",
    "FatTree16": "FT16",
    "FatTree32": "FT32",
    'Bellcanada': 'BC',
    'Cogentco': 'CG',
    'rf1239_real_hard': 'SL',
}
ns2idx = {
    "FatTree8": 0,
    "FatTree16": 1,
    "FatTree32": 2,
    'Bellcanada': 3,
    'Cogentco': 4,
    'rf1239_real_hard': 5,
}

# Plots

In [61]:
cols = [
    "Network Label",
    "SRAM Usage Mbits",
    "SRAM Usage Util",
    "# Alt Entries Total",
    "# Alt Entries /Node",
    "Planning Runtime"
]

pdf = pd.DataFrame(index=range(6), columns=cols)
for i, row in df.iterrows():
    pdf.iloc[ns2idx[i]] = [
        ns[i],
        f"{row['total_sram_Kbits']/1024:.2f}",
        f"{row['total_sram_block_util']:.2f} %",
        int(row['n_total_alt_forwarding_entries']),
        int(row['n_alt_forwarding_entries']),
        f"{row['ttc_alt_forwarding']:.3f} s",
    ]

pdf.to_csv("./results/scalability/tableIII-summary.csv", index=False)
display(pdf)

,Network Label,SRAM Usage Mbits,SRAM Usage Util,# Alt Entries Total,# Alt Entries /Node,Planning Runtime
0,FT8,0.66,0.18 %,28288,484,0.426 s
1,FT16,1.31,0.35 %,914432,4008,11.361 s
2,FT32,7.44,2.00 %,29335552,32464,880.716 s
3,BC,0.66,0.18 %,6922,369,0.203 s
4,CG,1.53,0.41 %,201676,4648,2.280 s
5,SL,1.97,0.53 %,636312,6521,7.100 s
